In [10]:
import tweepy 
from tqdm.notebook import tqdm
import pandas as pd
import re
import numpy as np
import time
from datetime import datetime, timezone, timedelta
pd.set_option('display.max_colwidth', 180)
from langdetect import detect
pd.set_option('display.max_rows', None)
api_key = ""
api_secret = ""
access_key = ""
access_secret = ""
bearer = ""
def has_number_suffix(string):
    suffixes = ['st', 'nd', 'rd', 'th']
    for word in string.split():
        if word[-2:] in suffixes and word[:-2].isdigit():
            return True
    return False
def has_youth(string):
    words = string.lower().split()
    for i in range(len(words)):
        if words[i].startswith('u'):
            if words[i][1:].isdigit():
                return True
    return False
def is_spam2(text):
    try:
        if detect(text) != 'en':
            return True
    except:
        return True
    if "home" not in text:
        return True
        
        
    # Define regular expressions for identifying spam patterns
    excessive_punctuation_regex = r'[!?.]{4,}'
    poor_spelling_or_grammar_regex = r'\b[A-Z]{2,}\b|\b\w{0,2}\b'
    spam_keywords_regex = r'\b(free|pm|channel|bt|deal|bbc|offer|dm|target|line|lines|over|under|corner|corners|booked|booking|less|am|hd|sky|women|woman|formation|channel|channels|money|win|prize|link|stream|links|streams|live)\b'

    # Apply regular expressions to text input
    has_excessive_punctuation = bool(re.search(excessive_punctuation_regex, text))
    has_spam_keywords = bool(re.search(spam_keywords_regex, text.lower()))
    
    num_digits = sum(c.isdigit() for c in text)
    has_lots_of_numbers = num_digits > 3
    has_no_numbers = num_digits <2
    
    # Count the number of words in the input text
    num_words = len(text.split())

    # Determine if the input text has less than the minimum required number of words
    has_few_words = num_words < 3
    
    pattern = r"\d{1,2}[ap]m"
    matches = re.findall(pattern, text)
    if(len(matches)>0):
        has_time = True
    else:
        has_time = False
        
    
    pattern = r"\d+m"
    matches = re.findall(pattern, text)
    if(len(matches)>0):
        has_price = True
    else:
        has_price = False
        
        
    
    pattern = r"\b\d{2,}\b"
    matches = re.findall(pattern, text)
    if(len(matches)>0):
        has_dd = True
    else:
        has_dd = False
        
    pattern = r"\b\d{2,}[a-zA-Z]*\b"
    matches = re.findall(pattern, text)
    if(len(matches)>0):
        has_dd_suffix = True
    else:
        has_dd_suffix = False
    
    too_long = False
    if len(text)>80:
        too_long=True
        



    # Return True if text matches any of the spam patterns, False otherwise
    return too_long or has_youth(text) or has_number_suffix(text) or has_dd_suffix or has_dd or has_price or has_time or has_excessive_punctuation or has_few_words or has_no_numbers or has_spam_keywords or has_lots_of_numbers

def is_spam(text):
    # Define rules for identifying spam
    has_excessive_punctuation = text.count('!') > 3 or text.count('?') > 3
    has_poor_spelling_or_grammar = any(word.isupper() for word in text.split()) or len(text.split()) < 3
    has_spam_keywords = any(keyword in text.lower() for keyword in ['channel','free', 'link','live','offer', 'money', 'win', 'prize','stream'])
    # Return True if text matches any of the spam rules, False otherwise
    return has_excessive_punctuation or has_poor_spelling_or_grammar or has_spam_keywords



def search_archive(searchTerms = [], retweets = False, numTweets = 1000, start = None, end = None):
    start_time = time.time()
    team_tweets = []
    s_query = ""
    for item in searchTerms:
        if s_query == "":
            s_query += item
        else: 
            s_query += f" {item }"
    query = f'({s_query})'
    if retweets == False:
        query += " -is:retweet "
    query += " lang:en"
    print(f"Search query: {query}")

    
    for i,response in tqdm(enumerate(tweepy.Paginator(client.search_all_tweets, 
                                     query = query,
                                     user_fields = ['username', 'public_metrics', 'location'],
                                     tweet_fields = ['created_at', 'geo', 'public_metrics', 'text'],
                                     expansions = ['author_id'],
                                     start_time = start,
                                     end_time = end,
                                  max_results=500).flatten(numTweets))):
        team_tweets.append(response)
        time.sleep(0.002)
    time.sleep(1)
    print(f'{time.time()- start_time} seconds')
    return team_tweets
def convert_datetime(date, time):
    # Combine date and time strings
    datetime_str = f"{date} {time}"
    
    # Parse datetime string to datetime object
    dt = datetime.strptime(datetime_str, "%d/%m/%Y %H:%M")
    
    # Convert datetime object to ISO 8601 string with timezone offset
    iso_datetime_str = dt.replace(tzinfo=timezone.utc).isoformat()
    
    return iso_datetime_str
def clean_tweet(tweet,team1,team2):
    # Remove hashtags, @mentions, URLs, and RTs
    tweet = re.sub(r'#\w+', ' ', tweet)
    tweet = re.sub(r'@\w+', ' ', tweet)
    tweet = re.sub(r'https?://\S+', ' ', tweet)
    tweet = re.sub(r'RT', ' ', tweet)
    tweet = re.sub(r'\n',' ', tweet)
    

    # Strip whitespace and punctuation
    tweet = tweet.strip()
    tweet = re.sub(r'[^\w\s]', ' ', tweet)
    tweet = re.sub(' +', ' ', tweet)

    # Convert to lowercase
    tweet = tweet.lower()
    tweet = tweet.replace(team1, "home")
    tweet = tweet.replace(team2, "away")


    return tweet

client = tweepy.Client(bearer_token=bearer,wait_on_rate_limit=True)
tweet_dicts={}
df_schedule = pd.read_csv('/Users/moksh/PL Predictions/PremDataFull/Datasets/2018-19.csv')[['Date',"Time",'HomeTeam',"AwayTeam"]]
random_rows = df_schedule
num_spam = 0
for i,row in tqdm(random_rows.iterrows()):
    date = row['Date']
    time_string = row['Time']

    time_obj = datetime.strptime(time_string, '%H:%M')
    new_time_obj = time_obj - timedelta(hours=1)
    new_time_string = new_time_obj.strftime('%H:%M')
    time_= new_time_string
    time_intial = '00:00'
    team = row['HomeTeam'].lower()
    team_other = row['AwayTeam'].lower()
    #make start day of game
    start=convert_datetime(date,time_intial)
    #make end hour before game time
    end= convert_datetime(date,time_)
    tweet_texts=[]
    raw_tweets = search_archive(searchTerms= [team,team_other],
                                retweets=False, 
                                numTweets=20000, 
                                start=start, 
                                end= end )
    print('Done Getting Tweets')
    tweet_texts= [clean_tweet(tweet.text, team,team_other) for tweet in raw_tweets]
    tweet_dates = [tweet.created_at for tweet in raw_tweets]
    
    new_tweets_texts = []
    new_tweets_dates =[]
    for k,tweet in enumerate(tweet_texts):
        if is_spam2(tweet):
            pass
        
        else:
            new_tweets_texts.append(tweet)
            new_tweets_dates.append(tweet_dates[k])
        if(len(new_tweets_texts)>10000):
            print('Filled 25')
            print(k)
            break            
            
    
    
    try:
        df = pd.DataFrame(new_tweets_texts)
        df[0] = df[0].apply(lambda x:clean_tweet(x,team,team_other))
        df['date'] = new_tweets_dates
        df['home_team'] = [team for i in range(len(df))]
        df['away_team'] = [team_other for i in range(len(df))]
    except:
        pass
    tweet_dicts['{}-{}'.format(team.lower(),date)] = df


0it [00:00, ?it/s]

Search query: (manchester utd leicester city) -is:retweet  lang:en


0it [00:00, ?it/s]

1.6615259647369385 seconds
Done Getting Tweets
Search query: (newcastle utd tottenham) -is:retweet  lang:en


0it [00:00, ?it/s]

...

1.3938651084899902 seconds
Done Getting Tweets
Search query: (fulham newcastle utd) -is:retweet  lang:en


0it [00:00, ?it/s]

1.5190248489379883 seconds
Done Getting Tweets
Search query: (watford west ham) -is:retweet  lang:en


0it [00:00, ?it/s]

4.603107929229736 seconds
Done Getting Tweets


In [11]:
full_df = pd.concat(tweet_dicts.values())

In [12]:
full_df.reset_index(drop=True,inplace=True)

In [13]:
full_df.columns = ['tweet','date','home','away']

In [14]:
full_df.to_csv('/Users/moksh/PL Predictions/Twitter/18tweets.csv')

In [15]:
len(full_df)

8887

In [16]:
full_df

,tweet,date,home,away
0,home vs away prediction utd 0 0 leicester,2018-08-10 18:38:02+00:00,manchester utd,leicester city
1,thanks for tweeting a score prediction home 1 0 away,2018-08-11 11:08:03+00:00,fulham,crystal palace
2,thanks for tweeting a score prediction home 0 1 away,2018-08-11 11:08:01+00:00,bournemouth,cardiff city
3,my prediction home 1 1 away gray for us but can t see us keeping a clean sheet,2018-08-11 13:23:12+00:00,watford,brighton
4,do you have it in you to predict the scoreline home away we predict a 1 1 draw,2018-08-11 11:40:54+00:00,watford,brighton
5,do you have it in you to predict the scoreline home away we predict a 1 1 draw,2018-08-11 11:40:27+00:00,watford,brighton
6,can t wait for away to be on last when we draw 0 0 with home,2018-08-11 10:22:44+00:00,watford,brighton
7,home 2 2 away and hove albion,2018-08-11 09:06:39+00:00,watford,brighton
8,home 1 vs 2 away,2018-08-11 13:59:57+00:00,huddersfield,chelsea
9,my prediction home 1 2 away,2018-08-11 13:59:12+00:00,huddersfield,chelsea
